In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import TimeSeriesSplit

## QUANTILE REGRESSION

In [2]:
data = pd.read_csv('non_nan.csv', )
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [3]:
def create_lags_cov(data, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)
    
    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs

In [4]:
def create_lags_returns(data, tickers, period):

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_tickers

In [5]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

In [6]:
# Creating lagged variables
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

data, lagged_covs = create_lags_cov(data, covs, period=period_covs)
data, lagged_tickers = create_lags_returns(data, tickers, period= period_returns)

# Creating dummy variables
data['Quater_dummy'] = 0
data['Quater_rippel'] = 0
data['Date'] = pd.to_datetime(data['Date'])
for index, row in data.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        data.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        data.loc[index, 'Quater_dummy'] = 1

In [7]:
data

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,NVO_lag1,NVO_lag2,NVS_lag1,NVS_lag2,PFE_lag1,PFE_lag2,ROG_lag1,ROG_lag2,Quater_dummy,Quater_rippel
0,2000-12-01,3.9,174.600,5.717143,-0.087123,-0.826852,2.331298,6.672740,5.420678,-0.380932,...,-9.882362,-4.279274,5.700307,-0.807726,2.604919,-3.860903,1.748252,13.043478,1,0
1,2001-01-01,4.2,175.600,5.240500,-0.066423,4.766324,-13.766945,-16.294179,-11.357522,-15.325723,...,-7.125809,-9.882362,13.379157,5.700307,4.026887,2.604919,12.886598,1.748252,1,0
2,2001-02-01,4.2,176.000,5.160952,-0.195020,-9.538746,4.143180,2.851534,4.509829,0.837527,...,12.824829,-7.125809,-4.290496,13.379157,-1.847877,4.026887,-5.022831,12.886598,0,0
3,2001-03-01,4.3,176.100,5.098947,-0.217255,-5.857225,5.615486,-6.323922,-9.828843,-3.186580,...,-0.350508,12.824829,-0.373563,-4.290496,-0.332146,-1.847877,-8.846156,-5.022831,1,0
4,2001-04-01,4.4,176.400,4.885455,0.756989,8.836250,-1.512969,-5.723911,10.300706,10.879156,...,-0.150755,-0.350508,-7.827520,-0.373563,-8.777164,-0.332146,-0.112520,-8.846156,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,8.619586,14.479974,1.295221,10.390734,3.090302,7.690670,9.446071,-53.665066,1,0
266,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,2.541749,8.619586,-0.110227,1.295221,-13.817335,3.090302,16.968326,9.446071,0,0
267,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,1.592445,2.541749,-7.172811,-0.110227,-7.286115,-13.817335,5.451681,16.968326,1,0
268,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,12.873250,1.592445,9.367574,-7.172811,0.566924,-7.286115,11.025813,5.451681,1,0


### Defining tick_loss function

In [8]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss

In [9]:
data.columns

Index(['Date', 'unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth',
       'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE',
       'ROG', 'inflation_change', 'unemp_change', 'treasury_yield_change',
       'unemployment_rate_lag1', 'unemployment_rate_lag2',
       'unemployment_rate_lag3', 'CPI_lag1', 'CPI_lag2', 'CPI_lag3',
       'treasury_yield_lag1', 'treasury_yield_lag2', 'treasury_yield_lag3',
       'GDP_growth_lag1', 'GDP_growth_lag2', 'GDP_growth_lag3',
       'SP500_return_lag1', 'SP500_return_lag2', 'SP500_return_lag3',
       'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'AZN_lag1',
       'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1', 'JNJ_lag2', 'LLY_lag1',
       'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1', 'NVO_lag2', 'NVS_lag1',
       

In [10]:
import statsmodels.api as sm
import numpy as np

def rolling_forecast_QR(data, dates, ticker, alpha, columns_to_exclude, horizon):
    t_loss = 0
    dta = data.copy()
    all_var_forecast = []

    for d in dates:
        train_df = dta[dta.Date <= d]
        test_df = dta[dta.Date > d]
        test_df = test_df.reset_index(drop=True)

        X_train = train_df.drop(columns=columns_to_exclude,axis=1)
        y_train = train_df[ticker]

        X_test = test_df.drop(columns=columns_to_exclude, axis=1)
        X_test = X_test.iloc[horizon-1]
        y_test = test_df[ticker][horizon-1]

        quantreg = sm.QuantReg(y_train, X_train)
        quantreg_results = quantreg.fit(q=alpha)

        VaR_forecast = quantreg_results.predict(X_test)
        all_var_forecast.append(VaR_forecast)

        r = y_test
        loss = tick_loss(alpha, r, VaR_forecast)
        t_loss += loss

    combinations = 13 - horizon
    t_loss = t_loss / combinations
    all_var_forecast = np.concatenate(all_var_forecast)

    return t_loss, all_var_forecast

# Naïve Models Specifications

## 1. Macros Only

We are only going to train the model with the lagged macroeconomic variables

In [11]:
data_macro = data.copy()
to_exclude = []

for t in tickers:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

to_exclude

['AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

### 1-step-ahead VaR: 1 month

In [12]:
dates1m = ['2022-05-01','2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']

In [13]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=1)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 0.5973984308307704 

AZN-1m-VaR-forecast: [ -8.42148532  -4.7117599   -8.73083435 -10.29276875 -10.40801662
 -12.07501134  -0.88210064  -4.35815879 -14.94508848  -9.21300088
 -10.71829988 -10.77965598] 

BMY-1m-loss: 0.575476041176639 

BMY-1m-VaR-forecast: [ -6.32739385 -10.08606125 -10.11298188 -17.98095261   0.50303324
 -10.30913906 -27.15542053 -13.15869627 -13.89185267 -11.12614839
 -14.05645782 -15.24167195] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 0.5455903436256422 

JNJ-1m-VaR-forecast: [ -4.10273981  -1.50701229  -5.01958536  -1.08544395  -9.15235362
 -10.03262881  -3.81811447  -7.45731708  -9.21308055  -9.4632912
 -13.60501796 -12.25406794] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-1m-loss: 1.5483088342923867 

PFE-1m-VaR-forecast: [-13.24923352  -8.21720109  -0.65715548  -6.597352   -13.31371882
 -15.26685799  -6.74056392  -9.44136652 -17.66492282 -11.17945784
 -11.6885489  -15.34363344] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 4.889701702297191 

ROG-1m-VaR-forecast: [-11.68330142 -41.8838649  -19.14112647 -21.78093244 -16.30590638
  -6.99553474 -36.8828301  -18.41611147 -16.53053263 -20.68271407
 -23.75015203 -32.15856596] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


- AZN-1m-loss: 0.5973984308307704
- BMY-1m-loss: 0.575476041176639
- JNJ-1m-loss: 0.5455903436256422 
- LLY-1m-loss: 1.6973569041256713
- MRK-1m-loss: 0.8044542059407872
- NVO-1m-loss: 0.8365805401165188 
- NVS-1m-loss: 0.9885674254982467
- PFE-1m-loss: 1.5483088342
- ROG-1m-loss: 4.889701702297191 

### 3-step-ahead VaR: 3 month

In [14]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [15]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=3)
    print(f'{t}-3m-loss: {loss} \n')
    print(f'{t}-3m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-3m-loss: 0.6558712384913222 

AZN-3m-VaR-forecast: [ -8.72119684 -10.29276934 -10.23471045 -12.0485803   -0.88210546
  -4.3960085  -15.45365667  -9.27960177 -10.78996759 -10.86165687] 

BMY-3m-loss: 0.6245673839696252 

BMY-3m-VaR-forecast: [-10.03766047 -19.0498058    0.5045923   -9.32897444 -26.03678694
 -13.15870975 -15.06373016 -12.4478426  -14.18965913 -15.47249668] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-3m-loss: 0.6193128828724277 

JNJ-3m-VaR-forecast: [ -5.051173    -0.45140973  -9.46584755 -10.1800587   -3.92214042
  -7.45732275  -9.21994637  -9.4056649  -13.68997191 -12.79339859] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-3m-loss: 1.8158332343375925 

PFE-3m-VaR-forecast: [ -0.65617648 -14.04799868 -13.39868611 -16.39951912  -7.00050793
  -9.59805536 -20.89187025 -10.95062498 -11.68853602 -15.40877533] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-3m-loss: 5.773933181625251 

ROG-3m-VaR-forecast: [-18.44401297 -14.33725342 -15.5146311   -4.37871854 -30.14298626
 -21.0347158  -16.56134413 -20.8277197  -23.66241338 -31.72183356] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


- AZN-3m-loss: 0.6558712384913222 
- BMY-3m-loss: 0.6245673839696252 
- JNJ-3m-loss: 0.6193128828724277
- LLY-3m-loss: 1.7019854680981807  
- MRK-3m-loss: 0.8076374513027862 
- NVO-3m-loss: 0.8295255641718853 
- NVS-3m-loss: 0.5515096324077937 
- PFE-3m-loss: 1.8158332343375925 
- ROG-3m-loss: 5.773933181625251 

### 6-steap-ahead VaR: 6 months

In [16]:
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']

In [17]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=6)
    print(f'{t}-6m-loss: {loss} \n')
    print(f'{t}-6m-VaR-forecast: {var_forecast} \n')

AZN-6m-loss: 0.6528273818943031 

AZN-6m-VaR-forecast: [-12.14925975  -0.36909286  -3.34354967 -14.81829204  -9.29345746
 -10.96601935 -10.86158297] 

BMY-6m-loss: 0.6850064355040465 

BMY-6m-VaR-forecast: [ -8.98235004 -27.85599974 -14.0499484  -15.06370721 -12.44784844
 -14.86609858 -17.80242843] 

JNJ-6m-loss: 0.3978593727362109 

JNJ-6m-VaR-forecast: [ -8.67691582  -2.86731133  -7.42978155  -8.84923086  -9.58231945
 -13.65929963 -12.75358491] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-6m-loss: 0.7931910762586328 

LLY-6m-VaR-forecast: [ -6.0059175   -0.5305359  -16.51193239 -11.64083651 -17.71936595
 -11.28807679 -14.90980049] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-6m-loss: 0.8015591342854774 

MRK-6m-VaR-forecast: [ -9.14040679 -14.05383327 -12.68524332 -18.91049929 -13.64527944
 -13.90047169 -18.90477262] 



...

NVO-6m-loss: 0.8988064189348007 

NVO-6m-VaR-forecast: [ -9.39063129 -10.52509995 -18.52195358 -13.3586004  -13.56893124
 -10.12555563  -8.36614307] 

NVS-6m-loss: 0.5635828681578909 

NVS-6m-VaR-forecast: [ -5.75078453  -3.43381957  -9.93235083  -8.02675412 -11.36021256
  -8.96639554  -7.48802523] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-6m-loss: 0.725394820095324 

PFE-6m-VaR-forecast: [ -8.92786397  -5.48135596 -12.34505161 -21.11246763 -11.24002033
 -13.01609104 -16.66076959] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-6m-loss: 6.429821679764937 

ROG-6m-VaR-forecast: [-15.1593173  -30.86377096 -21.45965092  -6.62286436 -24.3143873
 -16.52782388 -29.56504793] 



- AZN-6m-loss: 0.6528273818943031 
- BMY-6m-loss: 0.6850064355040465 
- JNJ-6m-loss: 0.3978593727362109 
- LLY-6m-loss: 0.7931910762586328
- MRK-6m-loss: 0.8015591342854774 
- NVO-6m-loss: 0.8988064189348007 
- NVS-6m-loss: 0.5635828681578909 
- PFE-6m-loss: 0.725394820095324 
- ROG-6m-loss: 6.429821679764937 

### 9-step-ahead VaR: 9 month

In [18]:
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']

In [19]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=9)
    print(f'{t}-9m-loss: {loss} \n')
    print(f'{t}-9m-VaR-forecast: {var_forecast} \n')

AZN-9m-loss: 0.7397345430072204 

AZN-9m-VaR-forecast: [-14.78150875  -9.54888288 -11.04618244 -10.75959376] 

BMY-9m-loss: 0.6391510833192557 

BMY-9m-VaR-forecast: [-15.31386633 -13.12365485 -15.0727407  -17.80239794] 

JNJ-9m-loss: 0.5043637860054009 

JNJ-9m-VaR-forecast: [ -8.98417368  -9.538356   -13.21639928 -12.62959423] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


LLY-9m-loss: 1.0230690112149592 

LLY-9m-VaR-forecast: [-13.98769522 -14.39134296 -13.66404456 -14.90978064] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


MRK-9m-loss: 0.8618936602469718 

MRK-9m-VaR-forecast: [-18.91052612 -13.64525356 -13.90046522 -18.53622121] 

NVO-9m-loss: 0.8072643165031784 

NVO-9m-VaR-forecast: [-14.32630077 -15.06558568 -10.78482138  -8.06976338] 

NVS-9m-loss: 0.5979695572185233 

NVS-9m-VaR-forecast: [ -7.87174883 -11.29422761  -8.81604225  -7.48801543] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


PFE-9m-loss: 0.5506122686499644 

PFE-9m-VaR-forecast: [-18.30801982  -9.88211087 -11.96740027 -17.52909263] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-9m-loss: 1.161796284334185 

ROG-9m-VaR-forecast: [-19.73177042 -22.50836628 -18.70140862 -19.20429064] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


- AZN-9m-loss: 0.7397345430072204 
- BMY-9m-loss: 0.6391510833192557
- JNJ-9m-loss: 0.5043637860054009 
- LLY-9m-loss: 1.0230690112149592 
- MRK-9m-loss: 0.8618936602469718 
- NVO-9m-loss: 0.8072643165031784 
- NVS-9m-loss: 0.5979695572185233 
- PFE-9m-loss: 0.5506122686499644 
- ROG-9m-loss: 1.161796284334185 

### 12-step-ahead VaR: 12 months

In [20]:
dates12m = ['2022-05-01']

In [21]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude, horizon=12)
    print(f'{t}-12m-loss: {loss} \n')
    print(f'{t}-12m-VaR-forecast: {var_forecast} \n')

AZN-12m-loss: 0.5242707428426207 

AZN-12m-VaR-forecast: [-10.67661861] 

BMY-12m-loss: 0.7886645503301303 

BMY-12m-VaR-forecast: [-18.46848488] 

JNJ-12m-loss: 0.3655078482444087 

JNJ-12m-VaR-forecast: [-12.58810601] 

LLY-12m-loss: 1.252130718799116 

LLY-12m-VaR-forecast: [-16.5547594] 

MRK-12m-loss: 0.7077045872531031 

MRK-12m-VaR-forecast: [-18.53618017] 

NVO-12m-loss: 0.1207232473382402 

NVO-12m-VaR-forecast: [-6.3823803] 

NVS-12m-loss: 0.006291455470021701 

NVS-12m-VaR-forecast: [-6.28747454] 

PFE-12m-loss: 0.8541371023298567 

PFE-12m-VaR-forecast: [-19.31982235] 

ROG-12m-loss: 1.169527555864101 

ROG-12m-VaR-forecast: [-25.55271067] 



- AZN-12m-loss: 0.5242707428426207 
- BMY-12m-loss: 0.7886645503301303 
- JNJ-12m-loss: 0.3655078482444087 
- LLY-12m-loss: 1.252130718799116 
- MRK-12m-loss: 0.7077045872531031 
- NVO-12m-loss: 0.1207232473382402 
- NVS-12m-loss: 0.006291455470021701
- PFE-12m-loss: 0.8541371023298567 
- ROG-12m-loss: 1.169527555864101  

# 2. Ticker data

In [22]:
data_stocks = data.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')
to_exclude2

['unemployment_rate',
 'unemployment_rate_lag1',
 'unemployment_rate_lag2',
 'unemployment_rate_lag3',
 'CPI',
 'CPI_lag1',
 'CPI_lag2',
 'CPI_lag3',
 'treasury_yield',
 'treasury_yield_lag1',
 'treasury_yield_lag2',
 'treasury_yield_lag3',
 'GDP_growth',
 'GDP_growth_lag1',
 'GDP_growth_lag2',
 'GDP_growth_lag3',
 'SP500_return',
 'SP500_return_lag1',
 'SP500_return_lag2',
 'SP500_return_lag3',
 'inflation_change',
 'inflation_change_lag1',
 'inflation_change_lag2',
 'inflation_change_lag3',
 'unemp_change',
 'unemp_change_lag1',
 'unemp_change_lag2',
 'unemp_change_lag3',
 'treasury_yield_change',
 'treasury_yield_change_lag1',
 'treasury_yield_change_lag2',
 'treasury_yield_change_lag3',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG',
 'Date']

### 1-step-ahead VaR: 1 month

In [23]:
for t in tickers:
    loss, var_forecast = rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=1)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 0.9246658669637693 

AZN-1m-VaR-forecast: [ -9.36771076  -8.13137597  -2.62882678  -9.59354468  -1.72839603
  -5.33447889 -17.55886985 -20.07621851  -2.30508953  -8.09234079
  -9.79748238 -11.79650003] 

BMY-1m-loss: 1.8087503597175576 

BMY-1m-VaR-forecast: [-13.82705149 -13.47864121  -2.25699303  -8.89940481  -6.34795734
  -9.13750069 -27.16562517 -18.5975089    5.13173218  -9.56104948
 -15.84115568  -8.21397313] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 1.4361016556024417 

JNJ-1m-VaR-forecast: [ -6.15649329  -7.81781753  -0.07232725  -4.33346428  -2.70630701
  -4.49657262 -14.93481871 -11.78481826   0.84192782  -0.29090706
  -4.72426797  -8.02053309] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

NVO-1m-loss: 1.377840118704824 

NVO-1m-VaR-forecast: [ -8.20196364 -12.41155919  -0.09273882  -9.0359561   -3.89089468
  -4.23598953 -20.74529059 -19.09984813   2.30348934  -9.1813527
 -11.11360099  -8.83932217] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-1m-loss: 1.791568422036246 

NVS-1m-VaR-forecast: [ -5.78553099 -11.63144006  -1.73271593  -7.9543678   -3.1623979
  -6.09390647 -17.7371067  -20.98813932   3.32509782  -5.01667869
  -4.84942     -5.92433106] 

PFE-1m-loss: 1.3525566069687258 

PFE-1m-VaR-forecast: [ -2.59299749 -11.40880616  -4.79890379 -12.26902203  -1.55699875
  -2.91433986 -14.63434576 -11.79033581  -1.06547875  -9.03582999
 -10.30013641  -5.41391124] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 5.831112464847229 

ROG-1m-VaR-forecast: [-18.52425012 -20.56218378   0.50279445 -21.71270189  -5.49670467
   1.58608746 -34.32139633 -39.97136489   4.11646439 -17.58537641
 -12.87910199  -6.4869154 ] 



### 3-step-ahead VaR: 3 months

In [24]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=3)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-1m-loss: 1.0758413471071586 

AZN-1m-VaR-forecast: [ -2.62579691  -9.10494199  -0.93597553  -5.58330016 -17.55886023
 -20.3700372   -1.8040118   -8.46628317 -11.25920174 -11.74367257] 

BMY-1m-loss: 2.0334773246449407 

BMY-1m-VaR-forecast: [ -2.25700077  -8.82525446  -7.23641161  -9.13750884 -26.83015557
 -18.68694508   5.08063517  -8.76117011 -15.57760286  -8.11929194] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 1.6408329005207105 

JNJ-1m-VaR-forecast: [ -0.14101453  -4.00853479  -2.52442805  -4.4927618  -14.93480162
 -11.78481852   0.80293103   0.84603873  -3.78793344  -7.95273913] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


...

NVO-1m-loss: 1.4786958881197552 

NVO-1m-VaR-forecast: [ -0.09280017  -9.03595952  -3.89077088  -4.23598732 -20.74518312
 -17.69442865   1.99892731  -8.61886289  -9.62869152  -8.83932106] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-1m-loss: 1.9258201223254718 

NVS-1m-VaR-forecast: [ -2.05368315  -7.57732382  -3.7224835   -6.34823796 -17.73712234
 -20.97157367   3.32509892  -3.95435462  -3.9456217   -5.92438122] 

PFE-1m-loss: 1.5120735723532128 

PFE-1m-VaR-forecast: [ -4.85362477 -12.0208297   -1.24671363  -2.90407094 -14.73048589
 -12.14804509  -1.29177635 -10.41976804  -9.32157885  -5.41391359] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 6.74533447908787 

ROG-1m-VaR-forecast: [  0.50279267 -20.03580684  -3.20141294   1.75055126 -21.7619532
 -43.55680943   4.11643652 -17.5853904  -12.87903768  -6.48683556] 



### 6-step-ahead VaR: 6 months

In [25]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=6)
    print(f'{t}-6m-loss: {loss} \n')
    print(f'{t}-6m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-6m-loss: 0.7370518875857861 

AZN-6m-VaR-forecast: [ -5.31214763 -17.4146239  -21.82716361  -0.15284737  -8.09146432
 -11.1052869  -11.74371174] 

BMY-6m-loss: 1.8365369346119886 

BMY-6m-VaR-forecast: [-10.9645861  -29.07099176 -18.36168434   5.06959038  -8.76151383
 -15.75317432  -6.48414468] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-6m-loss: 1.2296161416742797 

JNJ-6m-VaR-forecast: [ -4.35494076 -14.06263368 -12.63789673   0.79542515   0.84603414
  -3.86031112  -7.93130369] 

LLY-6m-loss: 1.7919037860229394 

LLY-6m-VaR-forecast: [ -3.63485244 -12.48745292  -9.85612078  -0.26778537  -3.1930903
  -6.60225483  -8.50107073] 

MRK-6m-loss: 0.8056362630956629 

MRK-6m-VaR-forecast: [ -5.66340318 -19.26712312 -17.70444068   0.54968688  -7.34026591
  -9.76473471  -9.84152342] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-6m-loss: 0.8557351878629481 

NVO-6m-VaR-forecast: [ -4.23599235 -20.74534442 -17.69400808   1.99899317  -9.10366992
 -11.04063568  -8.87532912] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-6m-loss: 2.1333123666452645 

NVS-6m-VaR-forecast: [ -6.41649707 -18.42481041 -21.41956396   3.4154374   -4.2649676
  -3.98393014  -5.86167016] 

PFE-6m-loss: 1.5199251649499268 

PFE-6m-VaR-forecast: [ -2.86981911 -13.34911894 -10.89045617  -1.31295692 -10.0836429
  -9.38143143  -3.57537412] 

ROG-6m-loss: 8.266701996624159 

ROG-6m-VaR-forecast: [ -0.91102225 -30.23066374 -45.88726764   1.15505222 -20.57300805
 -13.77621127  -6.48708513] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


### 9-step-ahead VaR: 9 months

In [26]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=9)
    print(f'{t}-9m-loss: {loss} \n')
    print(f'{t}-9m-VaR-forecast: {var_forecast} \n')

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


AZN-9m-loss: 0.9099641983914197 

AZN-9m-VaR-forecast: [  1.43893377  -5.17912081  -9.16301588 -12.25969144] 

BMY-9m-loss: 2.7574863904943823 

BMY-9m-VaR-forecast: [  5.83604035 -10.69317517 -16.13289529  -6.50430879] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-9m-loss: 1.8767077860402959 

JNJ-9m-VaR-forecast: [ 0.95332344  0.29439777 -4.06512847 -7.92985768] 

LLY-9m-loss: 3.0444106799475197 

LLY-9m-VaR-forecast: [ 0.50500719 -2.8189522  -6.37486071 -8.50120197] 

MRK-9m-loss: 1.0369129404819708 

MRK-9m-VaR-forecast: [ 1.73895347 -5.46575222 -8.85979985 -9.84257765] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-9m-loss: 0.6435526371436546 

NVO-9m-VaR-forecast: [  1.9989022   -9.1036492  -11.04047992  -8.87517304] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-9m-loss: 2.898665796322658 

NVS-9m-VaR-forecast: [ 3.11285351 -3.73065678 -3.40561605 -5.97073062] 

PFE-9m-loss: 1.7745312236442783 

PFE-9m-VaR-forecast: [ -0.64736341 -10.29763781  -8.29919839  -3.58090991] 

ROG-9m-loss: 0.4963888827241083 

ROG-9m-VaR-forecast: [  2.96030892 -16.43753924 -10.60151861  -2.8344949 ] 



### 12-step-ahead VaR: 12 months

In [27]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude2, horizon=12)
    print(f'{t}-12m-loss: {loss} \n')
    print(f'{t}-12m-VaR-forecast: {var_forecast} \n')

AZN-12m-loss: 0.5482028314997621 

AZN-12m-VaR-forecast: [-11.15526038] 

BMY-12m-loss: 0.12844730791612824 

BMY-12m-VaR-forecast: [-5.26414003] 

JNJ-12m-loss: 0.13525761193107164 

JNJ-12m-VaR-forecast: [-7.98310128] 

LLY-12m-loss: 0.8302012346069365 

LLY-12m-VaR-forecast: [-8.11616972] 

MRK-12m-loss: 0.276923926515625 

MRK-12m-VaR-forecast: [-9.92056696] 

NVO-12m-loss: 0.24537204461890597 

NVO-12m-VaR-forecast: [-8.87535625] 

NVS-12m-loss: 0.9614887701047898 

NVS-12m-VaR-forecast: [-5.14955198] 

PFE-12m-loss: 0.016139096474552053 

PFE-12m-VaR-forecast: [-2.55986223] 

ROG-12m-loss: 0.18238963546889583 

ROG-12m-VaR-forecast: [-5.80995226] 



# 3. All Covariates

In [28]:
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

to_exclude3

['Date',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'AZN',
 'BMY',
 'JNJ',
 'LLY',
 'MRK',
 'NVO',
 'NVS',
 'PFE',
 'ROG']

### 1-step-ahead VaR: 1 months

In [29]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates1m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=1)
    print(f'{t}-1m-loss: {loss} \n')
    print(f'{t}-1m-VaR-forecast: {var_forecast} \n')

AZN-1m-loss: 0.5079276964925706 

AZN-1m-VaR-forecast: [ -7.61857227  -0.37745581  -6.82325599 -11.48124416 -12.30741192
  -5.26026587  -1.93132624 -14.50824726 -17.25928955  -1.25552064
  -8.882634   -19.8281017 ] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-1m-loss: 0.5918539103580055 

BMY-1m-VaR-forecast: [-10.35690661 -12.32950473 -10.04379576   2.70476258 -14.22136722
 -18.03919733 -20.05837988 -13.32710257  -3.68305233 -13.64784223
 -19.39644759  -7.57230608] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-1m-loss: 2.1903887543716873 

JNJ-1m-VaR-forecast: [  3.37032639   0.91803463   8.82992516  -5.26546206  -6.95172854
 -12.98424934  -4.07035232 -16.09659276  -7.24953931 -13.50920163
 -13.1438927  -13.25714044] 

LLY-1m-loss: 1.7478643485190049 

LLY-1m-VaR-forecast: [-8.37916998 -1.4815798   0.09511685  4.94421541 -7.70725714 -9.84046692
 -1.6399168   0.93342853 -8.04841266  0.67033021 -7.64316883 -2.36775888] 

MRK-1m-loss: 0.6775421493440499 

MRK-1m-VaR-forecast: [ -4.13695358 -18.66440061 -22.10073919  -3.24794958  -7.30067813
 -10.66234938 -23.27108048 -13.31109346 -14.52088368  -5.12156205
  -3.72965318 -12.81150269] 



...

NVO-1m-loss: 0.7316751689588057 

NVO-1m-VaR-forecast: [-13.94838551 -13.17737452 -11.75927964 -17.31611138  -2.14032091
  -5.6686049  -20.64336122 -23.82618425  -5.09342573   1.00514371
  -5.54209451 -15.35681532] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-1m-loss: 1.500690054653104 

NVS-1m-VaR-forecast: [ -1.53477542  -4.41670162  -0.59798413  -4.2965583  -10.94808271
 -11.15681327  -3.92001253  -9.7571145   -6.65483055  -8.79170685
  -9.43009995 -16.78008953] 

PFE-1m-loss: 1.8012508922196033 

PFE-1m-VaR-forecast: [ -0.14624336  -7.91821671  -0.65558583 -16.2738069   -8.05322941
  -8.22203572  -4.02089354  -5.11708409 -15.31130981 -11.37201983
  -6.77635669 -15.72792169] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-1m-loss: 4.663419770167789 

ROG-1m-VaR-forecast: [  3.19609747 -32.34829619 -46.75804839 -17.44972878 -18.85442165
 -19.46996605  -9.89040781   7.91281354 -29.6618781  -29.37429799
 -18.58943582   6.21816457] 



### 3-step-ahead VaR: 3 months

In [30]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates3m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=3)
    print(f'{t}-3m-loss: {loss} \n')
    print(f'{t}-3m-VaR-forecast: {var_forecast} \n')

AZN-3m-loss: 0.5740648939718539 

AZN-3m-VaR-forecast: [ -6.82327351 -11.59327864 -12.2343114   -5.46480588  -2.12183846
 -14.55240479 -17.25928204  -1.3498097   -9.01061435 -19.6594751 ] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-3m-loss: 0.5598397171592046 

BMY-3m-VaR-forecast: [-10.02604033   6.07583502 -14.01888785 -15.90062215 -19.89720659
 -14.45579883  -4.65463676 -11.29192142 -18.93910149  -5.89287885] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-3m-loss: 1.8367115028653764 

JNJ-3m-VaR-forecast: [  7.10402377  -4.9087387   -6.03870955 -12.98425197  -3.94544182
 -15.38173261  -7.1526321  -12.95826439 -13.00838563 -13.37981351] 

LLY-3m-loss: 1.8550093332005038 

LLY-3m-VaR-forecast: [  0.04177761   6.85172676  -5.04285842  -9.98582483  -1.48977672
   0.9334401  -10.24895785   0.91720628  -7.75103211  -3.14811929] 

MRK-3m-loss: 0.7054914056731171 

MRK-3m-VaR-forecast: [-22.33990173  -4.60240131  -6.68551237 -10.64974942 -23.29837324
 -15.26611974 -10.2710136   -5.60008375  -3.96794267 -12.54616835] 



...

NVO-3m-loss: 0.7178340198250192 

NVO-3m-VaR-forecast: [ -9.38911658 -19.15370253  -3.78356226  -4.45775621 -20.88720329
 -20.00873588  -6.55626888  -1.09914016  -5.80181618 -15.38230973] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-3m-loss: 1.262869919886916 

NVS-3m-VaR-forecast: [  0.32941828  -9.07244404 -13.16413573 -11.67059964  -3.51299286
 -11.02465349  -6.48986113  -8.26815761  -9.51544557 -17.31322722] 

PFE-3m-loss: 2.1468503205591762 

PFE-3m-VaR-forecast: [ -0.80864032 -19.07902805  -6.63397819  -8.19034565  -3.61449308
  -4.89745285 -17.25898598 -11.93584595  -7.13470715 -15.72791699] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-3m-loss: 5.01293436214005 

ROG-3m-VaR-forecast: [-48.59053532 -17.550956   -18.74404524 -19.46998239  -2.8406381
  -3.9757574  -26.34415959 -29.4055971  -17.94075958   6.21818529] 



### 6-step-ahead VaR: 6 months

In [31]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates6m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=6)
    print(f'{t}-6m-loss: {loss} \n')
    print(f'{t}-6m-VaR-forecast: {var_forecast} \n')

AZN-6m-loss: 0.6671645066783346 

AZN-6m-VaR-forecast: [ -5.32547297  -1.78551199 -14.49058203 -17.14370876  -1.31984273
  -8.95704015 -19.59687661] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-6m-loss: 0.671867586937385 

BMY-6m-VaR-forecast: [-17.67479856 -19.86564139 -13.74067443  -3.59111655 -11.77559503
 -19.6583657   -8.1807429 ] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-6m-loss: 0.49400446494608424 

JNJ-6m-VaR-forecast: [ -6.97577252  -4.94737801 -19.38537564  -8.20739597 -12.65401393
 -12.22575849 -14.07510976] 

LLY-6m-loss: 1.347343727177321 

LLY-6m-VaR-forecast: [-10.9403455    0.57314039  -0.63924277 -10.57763856   0.77294802
  -7.73324543   0.22166189] 

MRK-6m-loss: 0.7096629044710789 

MRK-6m-VaR-forecast: [-11.31779488 -24.06179269 -15.15232415 -10.27112224 -10.64817208
  -4.94419676 -11.97963144] 



...

/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVS-6m-loss: 0.8461954772356453 

NVS-6m-VaR-forecast: [ -6.39711143  -0.20183797 -14.76240867  -5.53351886  -8.19466688
  -9.33990919 -17.30881489] 

PFE-6m-loss: 0.5009828012635555 

PFE-6m-VaR-forecast: [ -5.27220739 -10.70669235 -13.7271974  -17.0612012  -11.94171007
  -9.61755366 -16.68229135] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


ROG-6m-loss: 6.235768338067096 

ROG-6m-VaR-forecast: [-18.99388144  -3.65140049  -4.53347199 -25.86694805 -33.59593264
 -15.33259608   2.56943452] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


### 9-step-ahead VaR: 9 months

In [32]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates9m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=9)
    print(f'{t}-9m-loss: {loss} \n')
    print(f'{t}-9m-VaR-forecast: {var_forecast} \n')

AZN-9m-loss: 0.7588362296453004 

AZN-9m-VaR-forecast: [-17.4469945   -1.56948889  -9.04659518 -19.60122419] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


BMY-9m-loss: 0.42818306756635993 

BMY-9m-VaR-forecast: [ -4.66694447 -12.45338649 -19.51679855  -7.79808906] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


JNJ-9m-loss: 1.0955056592331194 

JNJ-9m-VaR-forecast: [ -3.51126687  -9.20223486 -11.40287586 -13.42652341] 

LLY-9m-loss: 0.4721735234067305 

LLY-9m-VaR-forecast: [-9.66265944  3.57999185 -7.21439137  0.4158346 ] 

MRK-9m-loss: 0.5245564185713537 

MRK-9m-VaR-forecast: [-10.99815251 -10.16545181  -4.86216381 -11.97971865] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +
/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-9m-loss: 0.609866873425034 

NVO-9m-VaR-forecast: [ -7.2883081   -1.13433981  -6.73997139 -17.29205646] 

NVS-9m-loss: 1.9951008035088333 

NVS-9m-VaR-forecast: [ -1.50036463  -7.60377843  -8.4465692  -16.24089709] 

PFE-9m-loss: 0.34474588866055356 

PFE-9m-VaR-forecast: [ -9.3393778   -7.66920177  -7.61958881 -16.58914482] 

ROG-9m-loss: 2.5412969971931787 

ROG-9m-VaR-forecast: [-24.40205621 -34.11069942 -15.30326422   3.86561485] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


### 12-step-ahead VaR: 12 months

In [33]:
for t in tickers:
    loss, var_forecast= rolling_forecast_QR(data, dates12m, t, alpha=0.05, columns_to_exclude= to_exclude3, horizon=12)
    print(f'{t}-12m-loss: {loss} \n')
    print(f'{t}-12m-VaR-forecast: {var_forecast} \n')

AZN-12m-loss: 0.968967870610338 

AZN-12m-VaR-forecast: [-19.57056116] 

BMY-12m-loss: 0.22741944423489163 

BMY-12m-VaR-forecast: [-7.24358276] 

JNJ-12m-loss: 0.5288247155125285 

JNJ-12m-VaR-forecast: [-15.85444335] 

LLY-12m-loss: 0.5231120245105833 

LLY-12m-VaR-forecast: [-1.97438552] 

MRK-12m-loss: 0.42708236796108534 

MRK-12m-VaR-forecast: [-12.92373578] 



/Users/lluisarull/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


NVO-12m-loss: 0.7084877764693764 

NVO-12m-VaR-forecast: [-18.13767089] 

NVS-12m-loss: 0.5254410833132236 

NVS-12m-VaR-forecast: [-16.67046709] 

PFE-12m-loss: 0.8340529954652492 

PFE-12m-VaR-forecast: [-18.91814021] 

ROG-12m-loss: 7.517352447826903 

ROG-12m-VaR-forecast: [5.75084303] 



### ABBV case

In [34]:
# ABBV case
data = pd.read_csv('non_nan.csv', )
data2 = pd.read_csv('monthly_data.csv')
data_ABBV = data2[['Date', 'ABBV']]
data_ABBV = data_ABBV.dropna()
data_ABBV

,Date,ABBV
158,2013-03-01,10.455062
159,2013-04-01,12.922975
160,2013-05-01,-6.438455
161,2013-06-01,-3.162299
162,2013-07-01,10.014486
...,...,...
276,2023-01-01,-8.576205
277,2023-02-01,5.152609
278,2023-03-01,3.554254
279,2023-04-01,-5.176628


In [35]:
merged_df = data_ABBV.merge(data, on='Date', how='inner')
merged_df

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2013-03-01,10.455062,7.5,232.282,1.984211,0.397212,3.337507,14.582205,11.414669,7.978126,4.851792,3.440210,-7.714279,9.067089,5.443922,-0.188640,-0.655,-0.2,0.069449
1,2013-04-01,12.922975,7.6,231.797,1.957500,0.085246,2.374537,3.881563,-3.568848,4.538222,-2.482836,7.359867,11.497419,3.537347,0.727651,-10.457790,-0.485,0.1,-0.026711
2,2013-05-01,-6.438455,7.5,231.893,1.759091,-0.190605,2.361004,-1.290451,16.816781,-1.231989,-4.008663,-0.638313,-8.792401,-2.711502,-6.329562,8.513136,0.096,-0.1,-0.198409
3,2013-06-01,-3.162299,7.5,232.445,1.928182,0.620501,-1.853781,-7.707318,-2.868894,2.762464,-6.788137,-0.535372,-3.805087,-1.463219,3.722003,2.269287,0.552,0.0,0.169091
4,2013-07-01,10.014486,7.3,232.900,2.300000,0.210359,5.724173,7.230424,-3.244595,8.898141,8.122938,4.655361,9.001745,1.272809,4.355584,17.540151,0.455,-0.2,0.371818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
119,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
120,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
121,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [36]:
merged_df.columns

Index(['Date', 'ABBV', 'unemployment_rate', 'CPI', 'treasury_yield',
       'GDP_growth', 'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO',
       'NVS', 'PFE', 'ROG', 'inflation_change', 'unemp_change',
       'treasury_yield_change'],
      dtype='object')

In [37]:
# Creating lagged variables
tickers_plus_ABBV = ['ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period_covs = [1, 2, 3]
period_returns = [1,2]

merged_df, lagged_covs = create_lags_cov(merged_df, covs, period=period_covs)
merged_df, lagged_tickers = create_lags_returns(merged_df, tickers_plus_ABBV, period= period_returns)

In [38]:
# Creating dummy variables
merged_df['Quater_dummy'] = 0
merged_df['Quater_rippel'] = 0
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
for index, row in merged_df.iterrows():
    if row['Date'].month in [3, 6, 9, 12]:
        merged_df.loc[index, 'Quater_dummy'] = 1
    if row['Date'].month in [1, 4, 7, 10]:
        merged_df.loc[index, 'Quater_dummy'] = 1

In [39]:
merged_df.columns

Index(['Date', 'ABBV', 'unemployment_rate', 'CPI', 'treasury_yield',
       'GDP_growth', 'SP500_return', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO',
       'NVS', 'PFE', 'ROG', 'inflation_change', 'unemp_change',
       'treasury_yield_change', 'unemployment_rate_lag1',
       'unemployment_rate_lag2', 'unemployment_rate_lag3', 'CPI_lag1',
       'CPI_lag2', 'CPI_lag3', 'treasury_yield_lag1', 'treasury_yield_lag2',
       'treasury_yield_lag3', 'GDP_growth_lag1', 'GDP_growth_lag2',
       'GDP_growth_lag3', 'SP500_return_lag1', 'SP500_return_lag2',
       'SP500_return_lag3', 'inflation_change_lag1', 'inflation_change_lag2',
       'inflation_change_lag3', 'unemp_change_lag1', 'unemp_change_lag2',
       'unemp_change_lag3', 'treasury_yield_change_lag1',
       'treasury_yield_change_lag2', 'treasury_yield_change_lag3', 'ABBV_lag1',
       'ABBV_lag2', 'AZN_lag1', 'AZN_lag2', 'BMY_lag1', 'BMY_lag2', 'JNJ_lag1',
       'JNJ_lag2', 'LLY_lag1', 'LLY_lag2', 'MRK_lag1', 'MRK_lag2', 'NVO_lag1

## 1. Macros Only

In [40]:
to_exclude = []

for t in tickers_plus_ABBV:
    to_exclude.append(t)
    to_exclude.append(f'{t}_lag1')
    to_exclude.append(f'{t}_lag2')

for covariate in covs:
    to_exclude.append(covariate)

to_exclude.append('Date')

In [41]:
to_exclude

['ABBV',
 'ABBV_lag1',
 'ABBV_lag2',
 'AZN',
 'AZN_lag1',
 'AZN_lag2',
 'BMY',
 'BMY_lag1',
 'BMY_lag2',
 'JNJ',
 'JNJ_lag1',
 'JNJ_lag2',
 'LLY',
 'LLY_lag1',
 'LLY_lag2',
 'MRK',
 'MRK_lag1',
 'MRK_lag2',
 'NVO',
 'NVO_lag1',
 'NVO_lag2',
 'NVS',
 'NVS_lag1',
 'NVS_lag2',
 'PFE',
 'PFE_lag1',
 'PFE_lag2',
 'ROG',
 'ROG_lag1',
 'ROG_lag2',
 'unemployment_rate',
 'CPI',
 'treasury_yield',
 'GDP_growth',
 'SP500_return',
 'inflation_change',
 'unemp_change',
 'treasury_yield_change',
 'Date']

In [42]:
# 1 month
loss, var_forecast = rolling_forecast_QR(merged_df, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude, horizon=1)
print('ABBV-1m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 3 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude, horizon=3)
print('ABBV-3m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 6 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude, horizon=6)
print('ABBV-6m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 9 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude, horizon=9)
print('ABBV-9m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 12 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude, horizon=12)
print('ABBV-12m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

ABBV-1m-loss: 4.782185318909551
ABBV-VaR-forecast: [  2.77583725 -26.93521843 -53.60174967   4.43696992 -25.35723454
 -19.46358922 -30.19267762   1.94465376 -35.00874661 -34.00877753
 -26.85143956  -9.47939362] 

ABBV-3m-loss: 4.514328864075574
ABBV-VaR-forecast: [-68.85500662  -3.01840718 -31.7276059  -21.18952869 -18.76223409
 -25.49171026 -30.80299228 -35.4299672  -27.16738699  -9.86774986] 

ABBV-6m-loss: 5.683615073735392
ABBV-VaR-forecast: [-20.9966639  -25.37303352 -20.67721695 -33.46762665 -23.03799114
 -21.20369739 -12.03464014] 

ABBV-9m-loss: 1.257969160266317
ABBV-VaR-forecast: [-31.7132149  -29.74639772 -20.52782803  -5.85222538] 

ABBV-12m-loss: 0.055897853650856935
ABBV-VaR-forecast: [-3.28011662] 



## 2. Ticker data

In [43]:
merged_df_stocks = merged_df.copy()
to_exclude2 = []

for covariate in covs:
    to_exclude2.append(covariate)
    to_exclude2.append(f'{covariate}_lag1')
    to_exclude2.append(f'{covariate}_lag2')
    to_exclude2.append(f'{covariate}_lag3')

for t in tickers:
    to_exclude2.append(t)

to_exclude2.append('Date')

In [44]:
# 1 month
loss, var_forecast = rolling_forecast_QR(merged_df, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude2, horizon=1)
print('ABBV-1m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 3 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude2, horizon=3)
print('ABBV-3m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 6 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude2, horizon=6)
print('ABBV-6m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 9 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude2, horizon=9)
print('ABBV-9m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 12 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude2, horizon=12)
print('ABBV-12m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

ABBV-1m-loss: 7.039900848504534
ABBV-VaR-forecast: [-17.05779787  -6.12790215   3.19600884 -17.15165135  13.66659465
  -5.97456171 -49.23385431  -7.07510827 -11.23153136 -11.91322051
 -12.39005317   3.51396232] 

ABBV-3m-loss: 9.196029743007568
ABBV-VaR-forecast: [  3.25900652 -19.11352723  13.35262281   2.81043908 -49.11451304
 -15.28728424 -15.02825911 -10.22287613 -12.39005681   3.51396201] 

ABBV-6m-loss: 8.877834841576348
ABBV-VaR-forecast: [  0.34578215 -46.00088927 -24.77626214  -4.98889052   2.43683167
 -16.01174152   2.36286083] 

ABBV-9m-loss: 0.7361173146709912
ABBV-VaR-forecast: [ -6.15400917  -3.01232673  -8.50473254 -28.42044995] 

ABBV-12m-loss: 0.7103550017157684
ABBV-VaR-forecast: [-16.36925958] 



## 3. All Covariates

In [45]:
merged_df_covs = merged_df.copy()
to_exclude3 = ['Date']

for covariate in covs:
    to_exclude3.append(covariate)

for t in tickers:
    to_exclude3.append(t)

In [46]:
# 1 month
loss, var_forecast = rolling_forecast_QR(merged_df, dates1m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=1)
print('ABBV-1m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 3 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates3m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=3)
print('ABBV-3m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 6 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates6m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=6)
print('ABBV-6m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 9 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates9m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=9)
print('ABBV-9m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

# 12 months
loss, var_forecast = rolling_forecast_QR(merged_df, dates12m, t, alpha=0.05, columns_to_exclude=to_exclude3, horizon=12)
print('ABBV-12m-loss:', loss)
print('ABBV-VaR-forecast:', var_forecast, '\n')

ABBV-1m-loss: 7.6361497537910905
ABBV-VaR-forecast: [  4.39953005  -2.70537872 -45.44563145  24.334506     1.49622455
  -6.82829454 -40.77738076  -0.91738769 -40.82931767 -22.37837885
 -24.93663504 -14.89061286] 

ABBV-3m-loss: 8.688662589879034
ABBV-VaR-forecast: [-49.14470758  24.7416907   -3.26911418  -3.72590457 -48.91466296
 -23.40584803 -31.63675074 -18.1861908  -30.26248506 -18.90092113] 

ABBV-6m-loss: 8.001989931526108
ABBV-VaR-forecast: [ -2.88457726 -33.96841251 -15.09819064 -22.66027755 -10.78522662
 -14.99585613 -18.72907641] 

ABBV-9m-loss: 0.8611929781566767
ABBV-VaR-forecast: [-23.1526601  -10.18325879 -15.17830576  -7.58334683] 

ABBV-12m-loss: 0.23149877259768045
ABBV-VaR-forecast: [-6.792135] 

